# Demo 2: Multiple projections
Below we project the CIFAR-10 dataset multiple times to different dimensions. This is a great use-case to demonstrate the speed of h-NNE, as most of the time is consumed in building a hierarchy while the actual projection is really fast.

In [4]:
import matplotlib.pyplot as plt
from torchvision.datasets import CIFAR10

from hnne import HNNE

## Load the CIFAR-10 train dataset
50000 examples of 32x32 RGB images

In [5]:
# Use this line to select the test set: cifar10 = CIFAR10(root='.', download=True, train=False)
cifar10 = CIFAR10(root='..', download=True, train=True)

In [6]:
cifar10_data = cifar10.data.reshape((len(cifar10.data), -1))
cifar10_data.shape

(50000, 3072)

## Project the same dataset to 2, 4, 8, 16, 32 and 62 dimensions
Once the HNNE object is initialized and the hierarchy has been build, one can run `fit_transform` multiple times setting different dimension with `override_n_components`.
The hierarchy will be built once and only thehnne projection will be applied for each different dimension.

In [14]:
%%time
hnne = HNNE(hnne_version="v2")
_ = hnne.fit_only_hierarchy(cifar10_data)

CPU times: user 29 s, sys: 3.5 s, total: 32.5 s
Wall time: 3.62 s


In [15]:
%%time
projections = []
for dim in [2, 4, 8, 16, 32]:
    print(f'Projecting to {dim} dimensions...')
    projection = hnne.fit_transform(cifar10_data, override_n_components=dim)
    projections.append(projection)

Projecting to 2 dimensions...
Projecting to 4 dimensions...
Projecting to 8 dimensions...
Projecting to 16 dimensions...
Projecting to 32 dimensions...
CPU times: user 5min 9s, sys: 2min 15s, total: 7min 24s
Wall time: 51.6 s


In [16]:
for projection in projections:
    print(projection.shape)

(50000, 2)
(50000, 4)
(50000, 8)
(50000, 16)
(50000, 32)


## Project new datasets to 2, 4, 8, 16, 32 and 62 dimensions
It could be desired to project multiple new datasets to different dimensions. One might be tempted to do this directly, which is not possible. One needs instead to re-project the original dataset to each dimension and then apply the `transform` method. Below is a code example.

In [17]:
# Use this line to select the test set: cifar10 = CIFAR10(root='.', download=True, train=False)
cifar10_test = CIFAR10(root='.', download=True, train=False)

In [18]:
cifar10_test_data = cifar10_test.data.reshape((len(cifar10_test.data), -1))
cifar10_test_data.shape

(10000, 3072)

In [19]:
%%time
projections_new_data = []
for dim in [2, 4, 8, 16, 32]:
    print(f'Projecting to {dim} dimensions...')
    # First re-build a projection of the original dataset to the desired dimension.
    _ = hnne.fit_transform(cifar10_data, override_n_components=dim)

    # The run transform on the new data.
    projection_new_data = hnne.transform(cifar10_test_data)
    
    projections_new_data.append(projection_new_data)

Projecting to 2 dimensions...
Projecting to 4 dimensions...
Projecting to 8 dimensions...
Projecting to 16 dimensions...
Projecting to 32 dimensions...
CPU times: user 5min 44s, sys: 2min 23s, total: 8min 7s
Wall time: 55.6 s


In [20]:
for projection in projections_new_data:
    print(projection.shape)

(10000, 2)
(10000, 4)
(10000, 8)
(10000, 16)
(10000, 32)
